In [1]:
import os
import zipfile
import shutil
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
extract_path = "/kaggle/input/plant-leaf/Plant_leave_diseases_dataset_without_augmentation"
output_dir = '/kaggle/working/'

In [3]:
# Create output subdirectories for train, valid, and test
subdirs = ['train', 'valid', 'test']
for subdir in subdirs:
    os.makedirs(os.path.join(output_dir, subdir), exist_ok=True)

# Split ratios
train_ratio = 0.7
valid_ratio = 0.2
test_ratio = 0.1

# Process each class folder
for class_name in os.listdir(extract_path):
    class_path = os.path.join(extract_path, class_name)
    
    # Check if it's a directory
    if os.path.isdir(class_path):
        # Get all image filenames in the folder
        images = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
        
        if not images:
            print(f"No images found in {class_name}, skipping...")
            continue
        
        # Split the dataset into train, valid, and test
        train_files, temp_files = train_test_split(images, test_size=(1 - train_ratio), random_state=42)
        valid_files, test_files = train_test_split(temp_files, test_size=(test_ratio / (valid_ratio + test_ratio)), random_state=42)
        
        # Copy files to respective directories
        for split, split_files in zip(subdirs, [train_files, valid_files, test_files]):
            split_class_dir = os.path.join(output_dir, split, class_name)
            os.makedirs(split_class_dir, exist_ok=True)
            
            for file_name in split_files:
                src = os.path.join(class_path, file_name)
                dest = os.path.join(split_class_dir, file_name)
                shutil.copy(src, dest)

print("Dataset has been successfully split into train, valid, and test sets!")




Dataset has been successfully split into train, valid, and test sets!


In [4]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths to split dataset directories
base_dir = "/kaggle/working/"
train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

# Image dimensions and batch size
img_height = 224
img_width = 224
batch_size = 32

# Data Generators for Training, Validation, and Test Sets (Only Rescaling)
data_gen = ImageDataGenerator(rescale=1.0/255.0)

# Loading Data with Data Generators
train_generator = data_gen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical' 
)

valid_generator = data_gen.flow_from_directory(
    valid_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = data_gen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  
    
)

# Display Class Indices
class_indices = train_generator.class_indices
print("Class Indices:", class_indices)


Found 38791 images belonging to 39 classes.
Found 11087 images belonging to 39 classes.
Found 5570 images belonging to 39 classes.
Class Indices: {'Apple___Apple_scab': 0, 'Apple___Black_rot': 1, 'Apple___Cedar_apple_rust': 2, 'Apple___healthy': 3, 'Background_without_leaves': 4, 'Blueberry___healthy': 5, 'Cherry___Powdery_mildew': 6, 'Cherry___healthy': 7, 'Corn___Cercospora_leaf_spot Gray_leaf_spot': 8, 'Corn___Common_rust': 9, 'Corn___Northern_Leaf_Blight': 10, 'Corn___healthy': 11, 'Grape___Black_rot': 12, 'Grape___Esca_(Black_Measles)': 13, 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 14, 'Grape___healthy': 15, 'Orange___Haunglongbing_(Citrus_greening)': 16, 'Peach___Bacterial_spot': 17, 'Peach___healthy': 18, 'Pepper,_bell___Bacterial_spot': 19, 'Pepper,_bell___healthy': 20, 'Potato___Early_blight': 21, 'Potato___Late_blight': 22, 'Potato___healthy': 23, 'Raspberry___healthy': 24, 'Soybean___healthy': 25, 'Squash___Powdery_mildew': 26, 'Strawberry___Leaf_scorch': 27, 'Strawberry

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization,
    GlobalAveragePooling2D)
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, roc_auc_score
import numpy as np

In [6]:
model = Sequential([
    # First block
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3), padding='same'),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    # Second block
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    # Third block
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    # Fourth block
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    # Global Average Pooling
    GlobalAveragePooling2D(),

    # Fully connected layers
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.25),
    Dense(len(train_generator.class_indices), activation='softmax')  # Number of classes
])

In [7]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [8]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=15,  
    verbose=1
)

Epoch 1/15


2024-12-06 05:53:14.999913: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1213/1213 [==============================] - 179s 140ms/step - loss: 1.1772 - accuracy: 0.6564 - val_loss: 1.3104 - val_accuracy: 0.6487
Epoch 2/15
1213/1213 [==============================] - 170s 140ms/step - loss: 0.6121 - accuracy: 0.8090 - val_loss: 2.3745 - val_accuracy: 0.4397
Epoch 3/15
1213/1213 [==============================] - 169s 139ms/step - loss: 0.3822 - accuracy: 0.8788 - val_loss: 0.9988 - val_accuracy: 0.7576
Epoch 4/15
1213/1213 [==============================] - 170s 140ms/step - loss: 0.2659 - accuracy: 0.9142 - val_loss: 0.4299 - val_accuracy: 0.8687
Epoch 5/15
1213/1213 [==============================] - 169s 139ms/step - loss: 0.2019 - accuracy: 0.9343 - val_loss: 0.6380 - val_accuracy: 0.8092
Epoch 6/15
1213/1213 [==============================] - 171s 141ms/step - loss: 0.1746 - accuracy: 0.9428 - val_loss: 0.4444 - val_accuracy: 0.8626
Epoch 7/15
1213/1213 [==============================] - 169s 140ms/step - loss: 0.1377 - accuracy: 0.9556 - val_loss: 0.241

In [9]:
test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)
print(f"Test Accuracy: {test_accuracy:.4f}")

175/175 [==============================] - 8s 47ms/step - loss: 0.7297 - accuracy: 0.8664
Test Accuracy: 0.8664


In [10]:
# Predictions and true labels
y_pred_probs = model.predict(test_generator, verbose=1)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true = test_generator.classes

# Metrics: Classification Report
print("\nClassification Report:")
class_labels = list(test_generator.class_indices.keys())
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

175/175 [==============================] - 9s 51ms/step

Classification Report:
                                               precision    recall  f1-score   support

                           Apple___Apple_scab       1.00      0.97      0.98        64
                            Apple___Black_rot       0.98      0.65      0.78        63
                     Apple___Cedar_apple_rust       0.70      1.00      0.82        28
                              Apple___healthy       0.88      0.92      0.90       165
                    Background_without_leaves       0.74      0.99      0.84       115
                          Blueberry___healthy       1.00      0.52      0.69       151
                      Cherry___Powdery_mildew       0.53      0.95      0.68       106
                             Cherry___healthy       0.99      0.79      0.88        86
   Corn___Cercospora_leaf_spot Gray_leaf_spot       0.69      0.92      0.79        52
                           Corn___Common_rust    